# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,beaudesert,-27.9833,152.9833,19.49,58,92,0.80,AU,1718866208
1,1,mammoth lakes,37.6485,-118.9721,10.75,41,0,3.60,US,1718866208
2,2,stanley,54.8680,-1.6985,13.77,79,0,4.63,GB,1718866132
3,3,raduzhny,62.0961,77.4750,13.16,32,0,1.12,RU,1718866209
4,4,novouzensk,50.4592,48.1431,26.94,43,51,3.99,RU,1718866209


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    scale = 1,
    size = "Humidity",
    color = "City",
    alpha=0.75,
    title="Humidity Map by City")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Ideal weather conditions
max_temp = 85
max_humidity = 100
max_cloudiness = 30
max_wind_speed = 12

# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) &
    (city_data_df["Humidity"] <= max_humidity) &
    (city_data_df["Cloudiness"] <= max_cloudiness) &
    (city_data_df["Wind Speed"] <= max_wind_speed)
]

ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,mammoth lakes,37.6485,-118.9721,10.75,41,0,3.60,US,1718866208
2,2,stanley,54.8680,-1.6985,13.77,79,0,4.63,GB,1718866132
3,3,raduzhny,62.0961,77.4750,13.16,32,0,1.12,RU,1718866209
5,5,hermanus,-34.4187,19.2345,7.67,76,8,4.16,ZA,1718866209
6,6,newman,37.3138,-121.0208,21.13,49,0,0.45,US,1718866210


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,mammoth lakes,US,37.6485,-118.9721,41,
2,stanley,GB,54.8680,-1.6985,79,
3,raduzhny,RU,62.0961,77.4750,32,
5,hermanus,ZA,-34.4187,19.2345,76,
6,newman,US,37.3138,-121.0208,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1  # Limits the results to n hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address =  response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, "City"]} - nearest hotel: {hotel_df.loc[index, "Hotel Name"]}")

# Display sample data
hotel_df

Starting hotel search
mammoth lakes - nearest hotel: Travelodge Mammoth Lakes
stanley - nearest hotel: Hotel 52
raduzhny - nearest hotel: Южная
hermanus - nearest hotel: Aloe guest house
newman - nearest hotel: Hampton Inn & Suites
constantia - nearest hotel: Hotel Dali
adamstown - nearest hotel: No hotel found
touros - nearest hotel: Pousada Atlântico
kolobrzeg - nearest hotel: Hotel Centrum
mezokovesd - nearest hotel: Zsóry Hotel
carnarvon - nearest hotel: No hotel found
serdar - nearest hotel: Wepa
snowflake - nearest hotel: Best Western
avarua - nearest hotel: Paradise Inn
whitehorse - nearest hotel: Town & Mountain Hotel
qarazhal - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
glendive - nearest hotel: La Quinta Inn & Suites Glendive
arraial do cabo - nearest hotel: No hotel found
lask - nearest hotel: Hotel Graf **
bilibino - nearest hotel: No hotel found
palmer - nearest hotel: La Quinta Inn & Suites
margaret river - nearest hotel: Margaret River 

,City,Country,Lat,Lng,Humidity,Hotel Name
1,mammoth lakes,US,37.6485,-118.9721,41,Travelodge Mammoth Lakes
2,stanley,GB,54.8680,-1.6985,79,Hotel 52
3,raduzhny,RU,62.0961,77.4750,32,Южная
5,hermanus,ZA,-34.4187,19.2345,76,Aloe guest house
6,newman,US,37.3138,-121.0208,49,Hampton Inn & Suites
...,...,...,...,...,...,...
567,miandoab,IR,36.9694,46.1027,21,هتل برنجیان
568,tanumah,SA,27.1000,44.1333,6,No hotel found
569,miquelon,PM,47.0975,-56.3814,98,Maison RFO
570,al jawf,SA,29.5000,38.7500,13,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    scale = 1,
    size = "Humidity",
    color = "City",
    alpha=0.75,
    title="Humidity Map by City",
    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)